In [30]:
import os
import numpy as np
import pandas as pd
import torch
import os.path as osp
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle
IMG_DIR = '/mnt/chicm/data/open-images/train/imgs'
MASK_DIR = '/mnt/chicm/data/open-images/masks/train'
DATA_DIR = '/mnt/chicm/data/open-images/segmentation'

In [31]:
def get_top_classes(start_index, end_index):
    df = pd.read_csv('top_classes_level1.csv')
    c = df['class'].values[start_index:end_index]
    #print(df.head())
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi

In [39]:
classes_0_20 = set(get_top_classes(0, 20)[0])
classes_20_100 = set(get_top_classes(20, 100)[0])
classes_100_275 = set(get_top_classes(100, 275)[0])
#classes_0_20

In [33]:
!ls $DATA_DIR

challenge-2019-classes-description-segmentable.csv
challenge-2019-label300-segmentable-hierarchy.json
challenge-2019-train-segmentation-bbox.csv
challenge-2019-train-segmentation-labels.csv
challenge-2019-train-segmentation-masks.csv
challenge-2019-validation-segmentation-bbox.csv
challenge-2019-validation-segmentation-labels.csv
challenge-2019-validation-segmentation-masks.csv
merge_0903_1.csv
merge_1000_0902.csv.zip
open-images-2019-instance-segmentation.zip
sample_empty_submission-Copy1.csv
sample_empty_submission.csv
sample_truncated_submission.csv
sub_0902_3_top50.csv.zip
sub1_first_1000_test_add_parent.csv
sub1_first_1000_test.csv
sub1_first_1000_test_th001.csv
sub1_first_1000_test_top50.csv
sub_first_0901_top50.csv.zip
sub_first_1000_0902_1_top50.csv
sub_first_1000_0902_2_top100.csv
sub_parent_0902_1000_1.csv
sub_parent_0903_1_top50.csv
test.zip


In [53]:
df = pd.read_csv(osp.join(DATA_DIR, 'sub_0902_3_top50.csv.zip'))
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0024876 eNqVU01vwyAM/UsGEjb1Ou3QJGC...
1,52a76f3fe21185bf,1024,683,/m/083wq 0.0176641 eNoLyIqyzDWISzFABob+2NmUAXS...
2,65e9da7c04c8e4cd,1024,682,NaN
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0065765 eNrNUttSwyAQ/SUICTTNxNFOLL...
4,0841dee573a875ca,1024,683,/m/05r655 0.0021448 eNqFlMmiqjoQRX/JlL4ZjcL0JK...


In [54]:
df.PredictionString = df.PredictionString.fillna('')
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0024876 eNqVU01vwyAM/UsGEjb1Ou3QJGC...
1,52a76f3fe21185bf,1024,683,/m/083wq 0.0176641 eNoLyIqyzDWISzFABob+2NmUAXS...
2,65e9da7c04c8e4cd,1024,682,
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0065765 eNrNUttSwyAQ/SUICTTNxNFOLL...
4,0841dee573a875ca,1024,683,/m/05r655 0.0021448 eNqFlMmiqjoQRX/JlL4ZjcL0JK...


In [55]:
#selected_rel = ['plays', 'inside_of', 'holds']
#selected_rel = 'on'
def get_filtered_pred_string(pred_str, selected_classes):
    if len(pred_str) < 1:
        return ''
    dets = []
    det = []
    for i, e in enumerate(pred_str.split(' ')):
        if i % 3 == 0:
            det = []
        det.append(e)
        if (i+1) % 3 == 0 and det[0] in selected_classes:
            dets.append(' '.join(det))
                
    return ' '.join(dets)

In [56]:
df.PredictionString = df.PredictionString.map(lambda x: get_filtered_pred_string(x, classes_20_100))

In [57]:
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/07r04 0.0049228 eNrNU0tvwyAM/kvh1Sw5RMphbdM...
1,52a76f3fe21185bf,1024,683,/m/01599 0.1188553 eNqlk9lymzAUhl9JCwjiSabT6Tg...
2,65e9da7c04c8e4cd,1024,682,
3,6d10fa21d93bd6cc,1024,682,/m/01n4qj 0.0017988 eNpLiA2zMwpJNTDxN4RAAwMYCw...
4,0841dee573a875ca,1024,683,/m/015qff 0.0016251 eNpVUGt3gjAM/UtpsfgY5Ux0Pi...


In [40]:
df.to_csv('sub_4195_20_100.csv', index=False)

In [24]:
!kaggle competitions submit -c open-images-2019-instance-segmentation -f sub_4195_20_100.csv -m "20_100"

100%|████████████████████████████████████████| 440M/440M [00:28<00:00, 16.0MB/s]
Successfully submitted to Open Images 2019 - Instance Segmentation

In [25]:
len(classes_100_276)

175

In [41]:
!ls

check_preds.ipynb	  sub_4195_100_275.csv	top_classes.csv
check_preds-parent.ipynb  sub_4195_100_276.csv	top_classes_level1.csv
eda.ipynb		  sub_4195_20_100.csv	top_classes_parent.csv
filter_classes.ipynb	  submit.ipynb
merge_df.ipynb		  submit-parent.ipynb


In [42]:
!md5sum sub_4195*

e28ce33501f39d8492276b1c4f4d9ca7  sub_4195_100_275.csv
e28ce33501f39d8492276b1c4f4d9ca7  sub_4195_100_276.csv
f6a517f5b5888dfa28f125ebddd39ac0  sub_4195_20_100.csv


# check top 5000

In [58]:
df = pd.read_csv('sub_4195_20_100.csv')
df_test = pd.read_csv(osp.join(DATA_DIR, 'sample_empty_submission.csv'))

In [59]:
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/07r04 0.0049228 eNrNU0tvwyAM/kvh1Sw5RMphbdM...
1,52a76f3fe21185bf,1024,683,/m/01599 0.1188553 eNqlk9lymzAUhl9JCwjiSabT6Tg...
2,65e9da7c04c8e4cd,1024,682,NaN
3,6d10fa21d93bd6cc,1024,682,/m/01n4qj 0.0017988 eNpLiA2zMwpJNTDxN4RAAwMYCw...
4,0841dee573a875ca,1024,683,/m/015qff 0.0016251 eNpVUGt3gjAM/UtpsfgY5Ux0Pi...


In [60]:
df_test.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,-1,-1,NaN
1,52a76f3fe21185bf,-1,-1,NaN
2,65e9da7c04c8e4cd,-1,-1,NaN
3,6d10fa21d93bd6cc,-1,-1,NaN
4,0841dee573a875ca,-1,-1,NaN


In [61]:
df_5000 = pd.concat([df.iloc[:5000].copy(), df_test.iloc[5000:].copy()])
df_5000.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/07r04 0.0049228 eNrNU0tvwyAM/kvh1Sw5RMphbdM...
1,52a76f3fe21185bf,1024,683,/m/01599 0.1188553 eNqlk9lymzAUhl9JCwjiSabT6Tg...
2,65e9da7c04c8e4cd,1024,682,NaN
3,6d10fa21d93bd6cc,1024,682,/m/01n4qj 0.0017988 eNpLiA2zMwpJNTDxN4RAAwMYCw...
4,0841dee573a875ca,1024,683,/m/015qff 0.0016251 eNpVUGt3gjAM/UtpsfgY5Ux0Pi...


In [62]:
df_5000.tail()

,ImageID,ImageWidth,ImageHeight,PredictionString
99994,e40f388b3a4f80a6,-1,-1,NaN
99995,a21b4cd415d83b4d,-1,-1,NaN
99996,750b5d6e41c23bc9,-1,-1,NaN
99997,4944489aca12f92e,-1,-1,NaN
99998,632dad5b912b4b6a,-1,-1,NaN


In [63]:
df_5000.to_csv('sub_4195_20_100_5000.csv', index=False)

In [ ]:
!kaggle competitions submit -c open-images-2019-instance-segmentation -f sub_4195_20_100_5000.csv -m "test 5000"

 77%|█████████████████████████████▏        | 18.6M/24.2M [00:07<01:02, 93.5kB/s]